# 02_ocr_testing.ipynb
## Pruebas de Extracción OCR

Objetivo: Prototipar y validar OCR (Landing AI vs DeepSeek)

In [2]:
import sys
import logging
from pathlib import Path
import json

# --- Configuración de Path ---
# Añadir el directorio PADRE (la raíz del proyecto) al path
# Esto es crucial porque el notebook está en 'notebooks/' y 'src' está en el nivel superior.
# .../Agente-Multimodal-Liquidador/
#    ├── notebooks/ (Aquí está el notebook)
#    └── src/       (Esto es lo que queremos importar)
#
# Path.cwd() -> .../notebooks
# Path.cwd().parent -> .../ (la raíz del proyecto)
#
# Así, Python puede encontrar 'src'
PROJECT_ROOT = str(Path.cwd().parent)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# --- Importaciones del Proyecto ---
# Ahora que el __init__.py de 'extractors' está corregido, podemos importar todo.


# Importa utilidades
from src.utils.logger import get_logger
from src.utils.config import (
    EXCEL_IMAGES_DIR, 
    PDF_IMAGES_DIR, 
    EXTRACTED_TEXT_DIR,     # Lo usaremos para probar el parser
    EXTRACTED_TABLES_DIR    # Para ver la salida del parser
)

# --- Configuración de Logging ---
# Configura el logger para que puedas ver los mensajes INFO en la salida del notebook
logger = get_logger(__name__)
logging.basicConfig(level=logging.INFO) 

print("✅ Dependencias del proyecto importadas correctamente.")
print(f"Raíz del proyecto establecida en: {PROJECT_ROOT}")

✅ Dependencias del proyecto importadas correctamente.
Raíz del proyecto establecida en: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent


## 1. Verificar Imágenes Disponibles

In [3]:
# Listar imágenes disponibles
excel_images = list(EXCEL_IMAGES_DIR.glob("*.png"))
pdf_images = list(PDF_IMAGES_DIR.glob("*.png"))
all_images = excel_images + pdf_images

print(f"Imágenes de Excel: {len(excel_images)}")
print(f"Imágenes de PDF: {len(pdf_images)}")
print(f"Total: {len(all_images)}")

if all_images:
    print(f"\nPrimeras imágenes:")
    for img in all_images[:5]:
        print(f"  - {img.name}")
else:
    print("\n⚠️ No hay imágenes. Ejecuta notebook 01 primero.")

Imágenes de Excel: 0
Imágenes de PDF: 1
Total: 1

Primeras imágenes:
  - 25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR_page_1.png


## 2. Inicializar OCR Extractor

In [5]:
# Crear extractor
# Cambiar provider en .env: OCR_PROVIDER=landing_ai o deepseeker
from src.extractors import OCRExtractor

extractor_landing = OCRExtractor(provider="landing_ai")
print("✅ Landing AI extractor inicializado")


2025-11-07 22:27:58,538 - src.extractors.ocr_extractor - INFO - Extractor OCR iniciado con el proveedor: landing_ai


INFO:src.extractors.ocr_extractor:Extractor OCR iniciado con el proveedor: landing_ai


2025-11-07 22:27:58,541 - src.extractors.ocr_extractor - ERROR - Falta LANDING_AI_API_KEY en el archivo .env


ERROR:src.extractors.ocr_extractor:Falta LANDING_AI_API_KEY en el archivo .env


✅ Landing AI extractor inicializado


## 3. Prueba OCR: Landing AI

In [6]:
# Probar con primera imagen
if excel_images:
    test_image = excel_images[0]
    print(f"Testing Landing AI con: {test_image.name}")
    
    result_landing = extractor_landing.extract_text(str(test_image))
    
    print(f"\nResultado:")
    print(f"  Status: {result_landing.get('status')}")
    print(f"  Provider: {result_landing.get('provider')}")
    
    if result_landing.get('status') == 'success':
        text = result_landing.get('text', '')
        print(f"  Texto extraído (primeros 200 chars):")
        print(f"  {text[:200]}...")
        print(f"  Total chars: {len(text)}")
    else:
        print(f"  ⚠️ Error: {result_landing.get('message')}")

## 4. Prueba OCR: DeepSeek

In [ ]:
# Probar con DeepSeek
if excel_images:
    test_image = excel_images[0]
    print(f"Testing DeepSeek con: {test_image.name}")
    
    result_deepseek = extractor_deepseek.extract_text(str(test_image))
    
    print(f"\nResultado:")
    print(f"  Status: {result_deepseek.get('status')}")
    print(f"  Provider: {result_deepseek.get('provider')}")
    
    if result_deepseek.get('status') == 'success':
        text = result_deepseek.get('text', '')
        print(f"  Texto extraído (primeros 200 chars):")
        print(f"  {text[:200]}...")
        print(f"  Total chars: {len(text)}")
    else:
        print(f"  ⚠️ Error: {result_deepseek.get('message')}")

## 5. Extracción de Estructura

In [ ]:
# Analizar estructura del texto extraído
if excel_images and 'result_landing' in locals():
    extracted_text = result_landing.get('text', '')
    
    structure = extractor_landing.extract_structure(extracted_text)
    
    print("Estructura detectada:")
    print(f"  Líneas: {len(structure.get('lines', []))}")
    print(f"  Tablas detectadas: {len(structure.get('tables', []))}")
    
    key_fields = structure.get('key_fields', {})
    print(f"\n  Campos clave:")
    print(f"    - Fechas: {key_fields.get('dates', [])}")
    print(f"    - Montos: {key_fields.get('amounts', [])}")
    print(f"    - Conceptos: {len(key_fields.get('concepts', []))}")

## 6. Parser: Estructuración JSON

In [ ]:
import json
from langchain_openai import ChatOpenAI
from src.extractors import StructureParser, LiquidacionData
from src.utils.config import LLM_MODEL

# --- 1. Inicializar el LLM y el Parser ---
# El StructureParser necesita una instancia de un LLM para funcionar.
# (Asegúrate de que tu OPENAI_API_KEY esté en el .env)
try:
    llm = ChatOpenAI(model=LLM_MODEL, temperature=0)
    
    # ¡CORRECCIÓN 1!
    # Le pasamos el 'llm' y el 'schema' (LiquidacionData) al inicializar.
    parser = StructureParser(llm=llm, schema=LiquidacionData)
    
    print(f"✅ Parser inicializado con modelo: {LLM_MODEL}")

except Exception as e:
    print(f"❌ Error inicializando el LLM. ¿Está la API Key en tu .env? Error: {e}")
    parser = None


# --- 2. Ejecutar el Parseo ---
# Asegúrate de haber ejecutado la celda anterior "Prueba OCR: Landing AI"
# para que 'result_landing' exista.

if parser and 'result_landing' in locals() and result_landing.get('status') == 'success':
    extracted_text = result_landing.get('text', '')
    
    if not extracted_text:
        print("⚠️ El texto extraído de Landing AI está vacío.")
    else:
        print(f"\nIniciando parseo con LLM (esto puede tardar unos segundos)...")
        
        try:
            # ¡CORRECCIÓN 2!
            # El método se llama 'parse_document()', no 'parse()'.
            structured_data = parser.parse_document(extracted_text)
            
            print("\n--- ✅ ¡Parseo Exitoso con LLM! ---")
            # Imprimimos el JSON completo. Es la mejor forma de ver la estructura.
            print(json.dumps(structured_data, indent=2, ensure_ascii=False))
            
            print("\n--- Acceso a datos ---")
            # Podemos acceder a los datos usando las claves del Pydantic (LiquidacionData)
            print(f"Cliente: {structured_data.get('cliente_nombre')}")
            
            # Usamos .get() con un diccionario vacío {} como default para evitar errores
            total = structured_data.get('resumen_financiero', {}).get('total_general')
            print(f"Total General: {total}")

        except Exception as e:
            # Esto atrapará errores de Pydantic (Validación) o del API de OpenAI
            print(f"\n--- ❌ ¡Error en el Parseo con LLM! ---")
            logger.error(f"Fallo el parseo con LLM: {e}", exc_info=True)
            print(str(e))
            
else:
    if not parser:
        print("⚠️ Parser no inicializado. Revisa el error de API Key.")
    else:
        print("⚠️ No se encontró 'result_landing' o la extracción OCR falló.")
        print("   Asegúrate de ejecutar la celda '3. Prueba OCR: Landing AI' primero.")

## 7. Comparación: Landing AI vs DeepSeek

In [ ]:
# Comparar calidad de extracción
if 'result_landing' in locals() and 'result_deepseek' in locals():
    text_landing = result_landing.get('text', '')
    text_deepseek = result_deepseek.get('text', '')
    
    print("COMPARACIÓN: Landing AI vs DeepSeek")
    print("="*60)
    print(f"\nLanding AI:")
    print(f"  - Status: {result_landing.get('status')}")
    print(f"  - Caracteres: {len(text_landing)}")
    print(f"  - Líneas: {len(text_landing.split(chr(10)))}")
    
    print(f"\nDeepSeek:")
    print(f"  - Status: {result_deepseek.get('status')}")
    print(f"  - Caracteres: {len(text_deepseek)}")
    print(f"  - Líneas: {len(text_deepseek.split(chr(10)))}")
    
    # Similitud aproximada
    if text_landing and text_deepseek:
        common_chars = len(set(text_landing) & set(text_deepseek))
        similarity = common_chars / max(len(text_landing), len(text_deepseek)) * 100
        print(f"\nSimilitud: {similarity:.1f}%")
    print("="*60)

## 8. Procesar todas las imágenes

In [ ]:
# Procesar lote de imágenes
print("Procesando todas las imágenes con OCR...")
print("="*60)

results_summary = {
    'success': 0,
    'error': 0,
    'total': len(all_images)
}

for idx, image in enumerate(all_images[:5], 1):  # Primeras 5 para testing
    print(f"\n[{idx}/{min(5, len(all_images))}] {image.name}")
    
    result = extractor_landing.extract_text(str(image))
    
    if result.get('status') == 'success':
        text = result.get('text', '')
        print(f"  ✅ Extraído: {len(text)} caracteres")
        results_summary['success'] += 1
    else:
        print(f"  ❌ Error: {result.get('message')}")
        results_summary['error'] += 1

print(f"\n{'='*60}")
print(f"RESUMEN:")
print(f"  Exitosos: {results_summary['success']}")
print(f"  Errores: {results_summary['error']}")
print(f"  Total: {results_summary['total']}")

## 9. Guardar resultados OCR

In [ ]:
# Guardar ejemplo de OCR + estructura en JSON
if 'structured' in locals():
    output_file = Path('ocr_test_output.json')
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(structured, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Resultados guardados en: {output_file}")
    print(f"\nPrimeras líneas:")
    print(json.dumps(structured, ensure_ascii=False, indent=2)[:500])

## 10. Resumen y Conclusiones

In [ ]:
print("="*60)
print("RESUMEN: EXTRACCIÓN OCR")
print("="*60)
print(f"\n✅ Imágenes procesadas: {len(all_images)}")
print(f"\n✅ Providers testeados:")
print(f"   - Landing AI (Enterprise)")
print(f"   - DeepSeek (Económico)")
print(f"\n✅ Estructura detectada:")
print(f"   - Fechas")
print(f"   - Montos (S/, USD)")
print(f"   - Conceptos")
print(f"   - Tablas")
print(f"\n✅ Próximo paso:")
print(f"   → Notebook 03: CLIP Embeddings & Espacio Compartido")
print("="*60)